In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [2]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Ikhlas/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Ikhlas/metadata_112.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,2_1.wav,1,112,Al-Ikhlas
1,2_2.wav,1,112,Al-Ikhlas
2,2_3.wav,1,112,Al-Ikhlas
3,2_4.wav,1,112,Al-Ikhlas
4,1_1.wav,2,112,Al-Ikhlas
5,1_2.wav,2,112,Al-Ikhlas
6,1_3.wav,2,112,Al-Ikhlas
7,1_4.wav,2,112,Al-Ikhlas
8,1.wav,3,112,Al-Ikhlas
9,2.wav,3,112,Al-Ikhlas


Feature Extraction

In [3]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [4]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [5]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.00065982 0.00096187 0.00105611 ... 0.36643252 0.3142157  0.27765784] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.09734584  0.09352862  0.09495907 ... -0.0624644  -0.05461849
 -0.05344357] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.27445894 0.22572826 0.20558178 ... 0.16945887 0.07598746 0.02161639] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.0495293  -0.04057811 -0.0735

In [6]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (4, 704, 40)
Shape of labels for fold 1: (4,)


In [7]:
mfcc_data_by_fold[3]["data"].shape

(4, 704, 40)

In [8]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [9]:
mfcc_data_by_fold[1]["data"].shape

(4, 704, 40, 1)

After Training Work, Deep Features 

In [10]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 56ms/step


In [11]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [12]:
intermediate_layer_outputs.shape

(3, 4, 1024)

In [13]:
intermediate_layer_outputs

array([[[-0.7723583 ,  0.99977845,  0.9931204 , ...,  0.95230484,
         -0.996556  , -0.9989959 ],
        [ 0.27853984, -0.49541616,  0.19128628, ...,  0.43072549,
         -0.99350685,  0.48450845],
        [-0.42096034,  0.99675286,  0.99633896, ...,  0.928379  ,
         -0.76921886, -0.99782264],
        [-0.9273225 , -0.85938245,  0.99121815, ...,  0.9584141 ,
         -0.91810274, -0.9985679 ]],

       [[-0.97089106, -0.9442637 ,  0.99917907, ...,  0.9909407 ,
         -0.9765208 , -0.9986903 ],
        [ 0.12287705, -0.98120856,  0.99930054, ...,  0.79761904,
         -0.9191569 , -0.9985452 ],
        [-0.5937335 ,  0.9992535 ,  0.99373305, ...,  0.9432749 ,
         -0.9746527 , -0.9980872 ],
        [-0.9542508 ,  0.9978574 ,  0.9990384 , ...,  0.69193053,
         -0.7335738 , -0.9999748 ]],

       [[-0.6778858 ,  0.9966063 ,  0.9960319 , ...,  0.98029196,
         -0.9996476 , -0.9847367 ],
        [ 0.2428239 ,  0.9937839 ,  0.9755024 , ...,  0.8241374 ,
         -0.

In [14]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [15]:
al_ikhlas_mean = mean_across_folds

In [16]:
al_ikhlas_mean.shape

(4, 1024)

In [17]:
al_ikhlas_mean

array([[-0.80704504,  0.35070702,  0.99611044, ...,  0.9745125 ,
        -0.99090815, -0.9941409 ],
       [ 0.21474694, -0.16094695,  0.7220297 , ...,  0.68416065,
        -0.95771354, -0.4744148 ],
       [-0.23865885,  0.9986677 ,  0.9907732 , ...,  0.79172397,
        -0.9126661 , -0.9981494 ],
       [-0.5951215 ,  0.36564264,  0.99674845, ...,  0.82945657,
        -0.8778567 , -0.9994872 ]], dtype=float32)

In [18]:
# Specify the file path where you want to save the .npy file
file_path = 'al_ikhlas_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_ikhlas_mean)

Testing of Single Audio

In [19]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/ikhlas_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.00827779 0.01175364 0.01195718 ... 0.06391279 0.03385095 0.01194416] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.14268626 0.30537844 0.30296853 ... 0.14962141 0.15857066 0.16601749] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00916394 -0.03786713 -0.06071532 ... -0.08792417 -0.14527759
 -0.15586883] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 1.70106277e-01  1.54157698e-01  1.24129415e-01 .

In [20]:
mfcc_extracted_features_single

array([[[[ -328.34464  ],
         [  117.98322  ],
         [  -21.96954  ],
         ...,
         [   -4.6924276],
         [   -7.5491514],
         [    5.618508 ]],

        [[ -313.32214  ],
         [  123.52269  ],
         [  -26.504478 ],
         ...,
         [    1.152132 ],
         [   -7.100539 ],
         [    5.887247 ]],

        [[ -330.50635  ],
         [  126.29224  ],
         [  -26.654266 ],
         ...,
         [    3.8891613],
         [   -8.564703 ],
         [    7.454237 ]],

        ...,

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
        

In [21]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 134ms/step


In [22]:
intermediate_output

array([[ 0.12287717, -0.98120856,  0.99930054, ...,  0.7976193 ,
        -0.91915727, -0.9985453 ]], dtype=float32)

In [23]:
intermediate_output.shape

(1, 1024)

In [24]:
intermediate_layer_outputs

array([[[-0.7723583 ,  0.99977845,  0.9931204 , ...,  0.95230484,
         -0.996556  , -0.9989959 ],
        [ 0.27853984, -0.49541616,  0.19128628, ...,  0.43072549,
         -0.99350685,  0.48450845],
        [-0.42096034,  0.99675286,  0.99633896, ...,  0.928379  ,
         -0.76921886, -0.99782264],
        [-0.9273225 , -0.85938245,  0.99121815, ...,  0.9584141 ,
         -0.91810274, -0.9985679 ]],

       [[-0.97089106, -0.9442637 ,  0.99917907, ...,  0.9909407 ,
         -0.9765208 , -0.9986903 ],
        [ 0.12287705, -0.98120856,  0.99930054, ...,  0.79761904,
         -0.9191569 , -0.9985452 ],
        [-0.5937335 ,  0.9992535 ,  0.99373305, ...,  0.9432749 ,
         -0.9746527 , -0.9980872 ],
        [-0.9542508 ,  0.9978574 ,  0.9990384 , ...,  0.69193053,
         -0.7335738 , -0.9999748 ]],

       [[-0.6778858 ,  0.9966063 ,  0.9960319 , ...,  0.98029196,
         -0.9996476 , -0.9847367 ],
        [ 0.2428239 ,  0.9937839 ,  0.9755024 , ...,  0.8241374 ,
         -0.

Euclidian Distance with Mean

In [25]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_ikhlas_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_ikhlas_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_ikhlas_mean: {closest_index+1}")
print(f"Closest value in al_ikhlas_mean: {closest_value}")

Index of closest value in al_ikhlas_mean: 2
Closest value in al_ikhlas_mean: [ 0.21474694 -0.16094695  0.7220297  ...  0.68416065 -0.95771354
 -0.4744148 ]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")